In [22]:
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
# populate vectorstore
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# load documents
docs = [WebBaseLoader(url).load() for url in urls]
individual_docs = [item for doc in docs for item in doc]

# split text
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=254, chunk_overlap=50)
doc_split = splitter.split_documents(individual_docs)

# store in vectorstore
vectorstore = Chroma.from_documents(
  documents=doc_split,
  embedding=OpenAIEmbeddings(),
  collection_name="blog-posts"
)

retriever = vectorstore.as_retriever()

retriever_tool = create_retriever_tool(
  retriever,
  name="retrieve_blog_posts",
  description="Search and return information about blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs")
tools = [retriever_tool]

In [24]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph import add_messages

class AgentState(TypedDict):
  messages = Annotated[list[AnyMessage], add_messages]

In [25]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2")
tools = [retriever_tool]

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

def agent(state):
  """
  Agent that invokes model to generate response based on the current state. It also decides whether to call retriever
  """
  print("=== Call Agent ===")
  
  sys_msg = """
    You are an expert in LLM agent, prompt engineering, and adversarial attacks on LLMs.
  Given the following context generate an answer to the question: {question}
  
  At the bottom of the answer, source where that relevant information was found as a list.
  The source should only contain urls.
  """
  llm_with_retriever = llm.bind_tools(tools)
  response = llm_with_retriever.invoke(state['messages'])
  
  return {
    "messages": [response]
  }
 
def rewrite(state):
  """
  Transform the query to produce a better question
  """
  print("=== Rewrite Query ===")
  rewrite_query = f"""
  Look at the input and try to reason about the underlying semantic intent / meaning
  Here is the initial question:
  ----------
  {state['messages'][0].content}
  ----------
  Formulate an improved question
  """
  response = llm.invoke(HumanMessage(content=rewrite_query))
  return { 'messages': [response]}

def generate_answer(state):
  """
  Generate answer
  """
  
  print("=== Generate Answer ===")
  question = state['messages'][0].content
  
  sys_msg = f"""
  You are an expert in LLM agent, prompt engineering, and adversarial attacks on LLMs.
  Given the following context generate an answer to the question: {question}
  
  At the bottom of the answer, source where that relevant information was found as a list.
  The source should only contain urls.
  """
  
  response = llm.invoke([SystemMessage(content=sys_msg)] + state['messages'])
  
  return {
    'messages': [response]
  }

# edges

  

: 